# Rechenaufgaben zu 5.2 (Convolutional Neural Networks)

---

## Rechenaufgabe 1

Seien $M_I$ und $M_K$ wie folgt definiert:

$$
M_I= \begin{pmatrix}
1 & 2 & 3 & 4 & 5 \\
3 & 5 & 2 & 1 & 3 \\
5 & 7 & 10 & 2 & 7 \\
6 & 8 & 13 & 7 & 1 \\
10 & 12 & 10 & 9 & 8
\end{pmatrix} 
$$

$$
M_K=\begin{pmatrix}
1 & 0 & -1 \\
2 & 0 & -2 \\
1 & 0 & -1
\end{pmatrix}
$$

**Aufgabe:** Berechnen Sie die Faltung $M_I \star M_K$ ohne Padding und mit einem Stride von 1.


In [29]:
def frob_scalprod(a_mat: list[list[int]], b_mat: list[list[int]]) -> int:
    return sum(
        a * b for a_row, b_row in zip(a_mat, b_mat) for a, b in zip(a_row, b_row)
    )


def conv_without_padding_and_stride_1(input_mat, kernel_mat):
    kernel_height, kernel_width = len(kernel_mat), len(kernel_mat[0])
    input_height, input_width = len(input_mat), len(input_mat[0])

    result = []

    for i in range(input_height - kernel_height + 1):
        selected_input_rows = input_mat[i : i + kernel_height]
        sub_result = []
        for j in range(input_width - kernel_width + 1):
            sub_input_mat = [row[j : j + kernel_width] for row in selected_input_rows]
            frob_result = frob_scalprod(sub_input_mat, kernel_mat)
            sub_result += [frob_result]

        result += [sub_result]

    return result


M_I = [
    [1, 2, 3, 4, 5],
    [3, 5, 2, 1, 3],
    [5, 7, 10, 2, 7],
    [6, 8, 13, 7, 1],
    [10, 12, 10, 9, 8],
]

M_K = [
    [1, 0, -1],
    [2, 0, -2],
    [1, 0, -1],
]

conv_without_padding_and_stride_1(M_I, M_K)

[[-5, 11, -1], [-16, 15, 17], [-19, 10, 29]]

## Rechenaufgabe 2

Seien $M$ wie folgt definiert:

$$
M=\begin{pmatrix}
1 & 3 & 4 & 10\\
1 & 2 & 5 & 12\\
2 & 3 & 0 & 1\\
4 & 5 & 2 & 1
\end{pmatrix}
$$

**Aufgabe:** Wenden Sie $2\times 2$-Max-Pooling mit einem Stride von $2$ an.

In [30]:
M = [
    [1, 3, 4, 10],
    [1, 2, 5, 12],
    [2, 3, 0, 1],
    [4, 5, 2, 1],
]


def sub_max_pool(submatrix):
    return max([val for row in submatrix for val in row])


def max_pool(matrix, kernel_size, stride):
    matrix_height, matrix_width = len(matrix), len(matrix[0])
    result = []
    for i in range(0, matrix_height - 1, stride):
        sub_result = []
        for j in range(0, matrix_width - 1, stride):
            sub_pool_result = sub_max_pool(
                [row[j : j + kernel_size] for row in matrix[i : i + kernel_size]]
            )
            sub_result += [sub_pool_result]

        result += [sub_result]

    return result


max_pool(M, 2, 2)

[[3, 12], [5, 2]]

## Rechenaufgabe 3

Angenommen, Sie haben als Eingabe für ein CNN ein RGB-Bild der Größe $994 \times 994$. Die Dimensionen dieser Eingabe sind also, die $3$ Farbkanäle eingerechnet, $3 \times 994 \times 994$.

In der Architektur des CNNs sei festgelegt, dass als erstes eine Faltung mit $9$ $3 \times 3$-Filtern (ohne Padding und mit Stride $1$) durchgeführt wird.

**Aufgaben:** 

1. Wie lauten nun die Dimensionen der Feature Map?
2. Nun wird auf der Feature Map eine $2 \times 2$-Pooling-Operation (mit Stride $2$) durchgeführt.
    - Wie lauten die Dimensionen der daraus resultierenden Feature Map?
3. Nun wird eine Faltung mit $7$ $5 \times 5$-Filtern (wieder ohne Padding und mit Stride $1$) durchgeführt.
    - Wie lauten die Dimensionen der resultierenden Feature Map?
4. Nun wird eine $4\times 4$-Pooling-Operation (mit Stride $4$) durchgeführt.
    - Wie lauten die Dimensionen der resultierenden Feature Map?

In [10]:
def calc_feature_map_dim(
    dims: tuple[int, int, int], number_of_kernels, kernel: tuple[int, int], stride
):
    channel, width, height = dims
    kernel_width, kernel_height = kernel

    feat_map_channels = channel * number_of_kernels
    feat_map_width = int((width - kernel_width) / stride) + 1
    feat_map_height = int((height - kernel_height) / stride) + 1

    return feat_map_channels, feat_map_width, feat_map_height


input_dims = (3, 994, 994)

first_feature_map_dims = calc_feature_map_dim(input_dims, 9, (3, 3), 1)

second_feature_map_dims = calc_feature_map_dim(first_feature_map_dims, 1, (2, 2), 2)

third_feature_map_dims = calc_feature_map_dim(second_feature_map_dims, 7, (5, 5), 1)

fourth_feature_map_dims = calc_feature_map_dim(third_feature_map_dims, 1, (4, 4), 4)


print(f"1.) Dimensionen der ersten Feature Map: {first_feature_map_dims}")
print(f"2.) Dimensionen der zweite Feature Map: {second_feature_map_dims}")
print(f"3.) Dimensionen der dritten Feature Map: {third_feature_map_dims}")
print(f"4.) Dimensionen der vierten Feature Map: {fourth_feature_map_dims}")

1.) Dimensionen der ersten Feature Map: (27, 992, 992)
2.) Dimensionen der zweite Feature Map: (27, 496, 496)
3.) Dimensionen der dritten Feature Map: (189, 492, 492)
4.) Dimensionen der vierten Feature Map: (189, 123, 123)
